In [72]:
import numpy as np
import matplotlib.pyplot as plt


class Function:
    def __init__(self, value, delt=0.001):
        self.value = value

        def grad(x):
            array = []
            for i in range(len(x)):
                x[i] += delt
                first_val = value(x)
                x[i] -= 2 * delt
                second_val = value(x)
                x[i] += delt
                array.append((first_val - second_val) / (2 * delt))
            return np.array(array)

        self.grad = grad


wolfe_cond_template = lambda c1, c2, x, func, gk: lambda a, b: ((func.value(x - ((a + b) / 2) * gk) <= (func.value(x) + c1 * ((a + b) / 2) * np.dot(gk, -gk))) and (np.dot(func.grad(x - ((a + b) / 2) * gk), -gk) >= c2 * np.dot(gk, -gk)))

wolfe_cond = lambda: ""


def grad_desc(func, lr, x, eps, is_wolfe = False):
    global wolfe_cond
    points = [x]

    while True:
        prev_x = x
        grad = func.grad(x)
        if is_wolfe:
            wolfe_cond = wolfe_cond_template(0.1, 0.9, x, func, grad)
        x = x - lr(Function(lambda a: func.value(x - a * grad)), is_wolfe) * grad
        points.append(x)
        if abs(func.value(x) - func.value(prev_x)) < eps:
            break

    return np.array(points)


def right_border_calc(func):
    right_start = 0.25
    zero = func.value(0)
    while zero >= func.value(right_start):
        right_start *= 2

    return right_start


def dichotomy(func, a_1, a_2, eps, delt, is_wolfe):
    cond = lambda a, b: abs(a - b) >= eps
    if is_wolfe:
        cond = wolfe_cond
    while cond(a_1, a_2):
        new_a_1 = (a_1 + a_2) / 2 - delt
        new_a_2 = (a_1 + a_2) / 2 + delt

        if func.value(new_a_2) > func.value(new_a_1):
            a_2 = new_a_2
        else:
            a_1 = new_a_1

    return (a_1 + a_2) / 2


In [ ]:

def example1(x):
    return (x[0] - 4) ** 2 / 10 + (5 - x[1]) ** 2


def lr_wrapper(eps, delt):
    return lambda func, is_wolfe: dichotomy(func, 0, right_border_calc(func), eps, delt, is_wolfe)


print(grad_desc(Function(example1), lr_wrapper(0.01, 0.0001), np.array((20., 20.)), 0.0001))
print(grad_desc(Function(example1), lr_wrapper(0.01, 0.0001), np.array((20., 20.)), 0.0001, True))